In [1]:
import lightgbm as lgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error


In [2]:
df_train = pd.read_csv("processed/processed_train_2.csv")
df_test = pd.read_csv("processed/processed_test_2.csv")
df_train

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan,price_doc
0,43,27.000000,4.0,12.559171,1879,1.909844,6.399244,2.105145,6.407578e+06,155572,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.668660e+06
1,34,19.000000,3.0,12.559171,1879,1.909844,6.399244,2.105145,9.589337e+06,115352,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.814010e+06
2,43,29.000000,2.0,12.559171,1879,1.909844,6.399244,2.105145,4.808270e+06,101708,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.523310e+06
3,89,50.000000,9.0,12.559171,1879,1.909844,6.399244,2.105145,1.258354e+07,178473,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.269391e+07
4,77,77.000000,4.0,12.559171,1879,1.909844,6.399244,2.105145,8.398461e+06,108171,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.582519e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30464,44,27.000000,7.0,9.000000,1975,2.000000,6.000000,3.000000,1.005305e+07,175518,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.170610e+06
30465,86,59.000000,3.0,9.000000,1935,4.000000,10.000000,3.000000,7.307411e+06,75377,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.422501e+07
30466,45,34.404467,10.0,20.000000,1879,1.000000,1.000000,1.000000,2.553630e+07,4001,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.754869e+06
30467,64,32.000000,5.0,15.000000,2003,2.000000,11.000000,2.000000,6.050065e+06,78616,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.308151e+07


In [3]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns=['price_doc']),
                                                 df_train['price_doc'], test_size=0.15, random_state=42)

In [4]:
## https://neptune.ai/blog/lightgbm-parameters-guide

In [5]:
from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "n_estimators": trial.suggest_int("n_estimators", 800, 1200),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0, 1),
        "bagging_freq": 1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-10-30 18:24:38,750] A new study created in memory with name: no-name-698c5a4f-4e01-405f-8f7b-df472a06e2d4
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8864492913112696, colsample_bytree=0.7292891314199127 will be ignored. Current value: feature_fraction=0.8864492913112696
[LightGBM] [Warning] min_data_in_leaf is set=59, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=59
[LightGBM] [Warning] bagging_fraction is set=0.9112454745493027, subsample=0.9122521994357183 will be ignored. Current value: bagging_fraction=0.9112454745493027
[LightGBM] [Warning] lambda_l2 is set=2.229449447868286, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.229449447868286
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 18:24:44,034] Trial 0 finished with value: 2653111.496233502 and parameters: {'n_estimators': 1152, 'boosting': 'gbdt', 'lambda_l2': 2.229449447868286, 'bagging_fraction': 0.9112454745493027, 'num_leaves': 565, 'feature_fraction': 0.8864492913112696, 'max_depth': 4, 'learning_rate': 0.004187587848234434, 'subsample': 0.9122521994357183, 'colsample_bytree': 0.7292891314199127, 'min_data_in_leaf': 59}. Best is trial 0 with value: 2653111.496233502.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.8368953192500224, colsample_bytree=0.07639779896178667 will be ignored. Current value: feature_fraction=0.8368953192500224
[LightGBM] [Warning] min_data_in_leaf is set=52, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=52
[LightGBM] [Warning] bagging_fraction is set=0.06697772855853035, subsample=0.38879502438126395 will be ignored. Current value: bagging_fraction=0.06697772855853035
[LightGBM] [Warning] lambda_l2 is set=9.638169946599145, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.638169946599145
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 18:24:54,009] Trial 1 finished with value: 5063542.70817733 and parameters: {'n_estimators': 1171, 'boosting': 'dart', 'lambda_l2': 9.638169946599145, 'bagging_fraction': 0.06697772855853035, 'num_leaves': 26, 'feature_fraction': 0.8368953192500224, 'max_depth': 28, 'learning_rate': 0.002022372163629401, 'subsample': 0.38879502438126395, 'colsample_bytree': 0.07639779896178667, 'min_data_in_leaf': 52}. Best is trial 0 with value: 2653111.496233502.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

[I 2023-10-30 18:44:19,763] Trial 47 finished with value: 2680709.8353226334 and parameters: {'n_estimators': 1050, 'boosting': 'gbdt', 'lambda_l2': 5.354028330937584, 'bagging_fraction': 0.8126658192858758, 'num_leaves': 496, 'feature_fraction': 0.979223789248514, 'max_depth': 5, 'learning_rate': 0.003208130251315147, 'subsample': 0.665584952727149, 'colsample_bytree': 0.784036351565008, 'min_data_in_leaf': 56}. Best is trial 21 with value: 2549413.6181684.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9432936910178845, colsample_bytree=0.945578138094481 will be ignored. Current value: feature_fraction=0.9432936910178845
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Warning] bagging_fraction is set=0.8669421694188698, subsample=0.5074098278779302 will be ignored. Current value: bagging_fraction=0.8669421694188698
[LightGBM] [Warning] lambda_l2 is set=3.016277230613636, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.016277230613636
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 18:44:59,970] Trial 48 finished with value: 2580768.5035512666 and parameters: {'n_estimators': 1115, 'boosting': 'gbdt', 'lambda_l2': 3.016277230613636, 'bagging_fraction': 0.8669421694188698, 'num_leaves': 940, 'feature_fraction': 0.9432936910178845, 'max_depth': 28, 'learning_rate': 0.010848963811765235, 'subsample': 0.5074098278779302, 'colsample_bytree': 0.945578138094481, 'min_data_in_leaf': 36}. Best is trial 21 with value: 2549413.6181684.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.8547780179213449, colsample_bytree=0.8901535529942564 will be ignored. Current value: feature_fraction=0.8547780179213449
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7500441391343291, subsample=0.7438130051417878 will be ignored. Current value: bagging_fraction=0.7500441391343291
[LightGBM] [Warning] lambda_l2 is set=4.01454113985347, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.01454113985347
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 18:45:53,806] Trial 49 finished with value: 2711446.8093136037 and parameters: {'n_estimators': 1083, 'boosting': 'dart', 'lambda_l2': 4.01454113985347, 'bagging_fraction': 0.7500441391343291, 'num_leaves': 428, 'feature_fraction': 0.8547780179213449, 'max_depth': 13, 'learning_rate': 0.015175162966046466, 'subsample': 0.7438130051417878, 'colsample_bytree': 0.8901535529942564, 'min_data_in_leaf': 20}. Best is trial 21 with value: 2549413.6181684.


In [7]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'n_estimators': 1111, 'boosting': 'gbdt', 'lambda_l2': 4.986326594579468, 'bagging_fraction': 0.6726362178929178, 'num_leaves': 903, 'feature_fraction': 0.9997009943799137, 'max_depth': 13, 'learning_rate': 0.0063106163502641725, 'subsample': 0.6300808893105623, 'colsample_bytree': 0.8077059623500861, 'min_data_in_leaf': 41}
Best RMSE: 2549413.6181684
